In [472]:
import os
import numpy as np
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

PassengerId : 각 승객의 고유 번호로 의미 없음

Survived : 생존 여부 (종속 변수)

Pclass : 티켓의 승선권 클래스 - 승객의 사회적, 경제적 지위를 알 수 있다. 클래스에 따라 좀 더 생존에 용이한 구조이지 않을까?

1st = Upper
2nd = Middle
3rd = Lower
Name : 이름 - 이름으로 딱히 의미가 있을까? 큰 의미 없을 듯.

Sex : 성별 - 성별에 따라 먼저 구조될 수도, 생존에 용이할 수 있을 듯하다.

Age : 나이 - 각 세대별(10대, 20대 또는 아이, 청소년, 성인 등)로 차이가 있을 수 있다.

SibSp : 동반한 Sibling(형제자매)와 Spouse(배우자)의 수 - 동반 여행자의 수에 따라 생존 차이가 있을 수도 있다.

Parch : 동반한 Parent(부모) Child(자식)의 수 - 위와 동일

Ticket : 티켓의 고유넘버 - 티켓 자체로 생존여부를 알 수는 없을 듯하다.

Fare : 티켓의 요금 - 비싼 요금의 티켓과 생존 여부에 상관이 있을 수도 있다. 하지만 Pclass가 이 변수를 categorical하게 만든 변수가 아닐까?

Cabin : 객실 번호 - 어떤 객실 번호 범위는 좀 더 갑판 등의 생존에 용이한 위치에 있을 수도 있을 듯하다.

Embarked : 승선한 항 - 큰 의미 없을 듯하다.

C = Cherbourg
Q = Queenstown
S = Southampton

In [473]:
train_data = pd.read_csv("/kaggle/input/titanic/train.csv")
test_data = pd.read_csv("/kaggle/input/titanic/test.csv")

sub_data = test_data.copy()

In [474]:
# 훈련 데이터의 상위 7개 데이터 펼쳐보기
train_data.head(7)

In [475]:
# 테스트 데이터의 상위 7개 데이터 펼쳐보기
test_data.head(7)

In [476]:
# 훈련 데이터의 NULL 값 확인
train_data.info()

In [477]:
# 테스트 데이터의 NULL 정보 확인
# Age 값과 Cabin 값의 NULL 정보를 확인할 수 있음
test_data.info()

> Survived

In [478]:
# 38 퍼센트의 사람이 생존했다는 것을 훈련 데이터에서 확인할 수 있다.
train_data['Survived'].value_counts(normalize=True)

In [479]:
# 생존 데이터에 대한 그래프
sns.countplot(train_data['Survived'])

> P CLASS

In [480]:
# 훈련 데이터에서 생존자들의 P class의 숫자가 낮음에 따라 생존율이 낮다는 것을 확인할 수 있다.
train_data['Survived'].groupby(train_data['Pclass']).mean()

In [481]:
# 생존자 그륩에서의 P Class 데이터의 시각화
sns.countplot(train_data['Pclass'], hue=train_data['Survived'])

> NAME

In [482]:
# 정규식을 사용하여, train_data에서 이름을 추출
train_data['Initial'] = train_data.Name.str.extract('([A-Za-z]+)\.')
test_data['Initial'] = test_data.Name.str.extract('([A-Za-z]+)\.')

In [483]:
# 훈련 데이터에서 추출한 이니셜의 개수 출력
train_data['Initial'].value_counts(normalize=False)

In [484]:
# 훈련 데이터의 생존자에서 확인할 수 있는 이니셜의 평균치
train_data['Survived'].groupby(train_data['Initial']).mean()

In [485]:
# 훈련 데이터의 성별과 추출한 이름(성)에 대한 정보 나열
pd.crosstab(train_data['Initial'], train_data['Sex'])

In [486]:
# 위의 시각화한 자료를 토대로 이니셜을 Miss, Mr, Mrs, 그 밖에는 Other로 대체
# 그리고 initial의 평균 값을 도출
train_data['Initial'].replace(['Mlle','Mme','Ms','Dr','Major','Lady','Countess','Jonkheer','Col','Rev','Capt','Sir','Don','Dona'],['Miss','Miss','Miss','Mr','Mr','Mrs','Mrs','Other','Other','Other','Mr','Mr','Mr','Mr'],inplace=True)

> Sex

In [487]:
# 훈련 데이터의 생존자 중, 성별의 비율
# 이건 당연하게도, 여성과 아이들이 먼저라는 유명한 항해사의 말을 참고해보았을 때 맞는 결과이다.
train_data['Survived'].groupby(train_data['Sex']).mean()

> Age

In [488]:
# 따온 이니셜에 따른 각 컬럼의 평균값을 도출한다.
train_data.groupby('Initial').mean()

In [489]:
# 훈련 데이터에서 Age의 평균 값을 도출하기
train_data.loc[(train_data.Age.isnull())&(train_data.Initial=='Mr'),'Age'] = 33
train_data.loc[(train_data.Age.isnull())&(train_data.Initial=='Miss'),'Age'] = 22
train_data.loc[(train_data.Age.isnull())&(train_data.Initial=='Master'),'Age'] = 5
train_data.loc[(train_data.Age.isnull())&(train_data.Initial=='Mrs'),'Age'] = 36
train_data.loc[(train_data.Age.isnull())&(train_data.Initial=='Other'),'Age'] = 46

> Embarked

In [490]:
# 훈련 데이터에서의 Embarked 분포도
train_data['Embarked'].value_counts()

In [491]:
# 생존한 사람들 중, Embarked에 대한 분포
train_data['Survived'].groupby(train_data['Embarked']).mean()

In [492]:
# 2개의 NULL 데이터는 과반수인 S로 채운다.
train_data['Embarked'].fillna('S',inplace=True)

> SibSp

In [493]:
# 생존자 중, 동반 인원이 있는 사람들의 생존 평균
train_data['Survived'].groupby(train_data['SibSp']).mean()

In [494]:
# 혼자 탔다고 생존율이 높고 한 것 아니니, 그리 좋은 양분은 될 수 없음
train_data['SibSp'].value_counts()

> Parch

In [495]:
# 비교적 고르게 분포한 데이터를 볼 수 있는데, 그다지 효율적으로 쓰이지는 않을 것 같은 데이터이다.
train_data['Survived'].groupby(train_data['Parch']).mean()

In [496]:
# 처음에는 혼자인 사람들의 생존율이 높을거라고 생각을 했지만, 위의 데이터를 확인했을 때 맞지 않는 것을 확인할 수 있었다. 그다지 효율적으로 쓰이지는 않을 것 같은 데이터이다.
train_data['Parch'].value_counts()

> Fare

In [497]:
# 티겟을 범위를 5등분
pd.qcut(train_data['Fare'], 3).value_counts()

In [498]:
# 티켓의 값에 따라 생존율이 올라가는 것을 볼 수 있다.
train_data['Survived'].groupby(pd.qcut(train_data['Fare'], 3)).mean()

In [499]:
# 티겟의 값을 5등분하여 P Class에 대한 분포도를 확인할 수 있었다.
pd.crosstab(pd.qcut(train_data['Fare'], 5), columns=train_data['Pclass'])

> Cabin

In [500]:
# Cabin의 데이터를 효율적으로 보기 위해서 앞 글자를 따온다.
train_data['Cabin_Initial'] = train_data['Cabin'].apply(lambda x: str(x)[0])

In [501]:
# 700개 다달하는 NULL 값을 가지고 있지만, 결측치가 너무 많아서 사용하기 힘들다.
train_data['Cabin_Initial'].value_counts()

> 작업 시작 

In [502]:
# 변화된 데이터 값 확인
train_data.info()

In [503]:
# 변화된 테스트 데이터의 확인
test_data.info()

In [504]:
# 테스트 데이터에도 똑같이 AGE의 평균 값을 넣어준다
test_data['Initial'].replace(['Mlle','Mme','Ms','Dr','Major','Lady','Countess','Jonkheer','Col','Rev','Capt','Sir','Don','Dona'],['Miss','Miss','Miss','Mr','Mr','Mrs','Mrs','Other','Other','Other','Mr','Mr','Mr','Mr'],inplace=True)
test_data.groupby('Initial').mean()

In [505]:
test_data.loc[(test_data.Age.isnull())&(test_data.Initial=='Mr'),'Age'] = 32
test_data.loc[(test_data.Age.isnull())&(test_data.Initial=='Miss'),'Age'] = 22
test_data.loc[(test_data.Age.isnull())&(test_data.Initial=='Master'),'Age'] = 7
test_data.loc[(test_data.Age.isnull())&(test_data.Initial=='Mrs'),'Age'] = 39
test_data.loc[(test_data.Age.isnull())&(test_data.Initial=='Other'),'Age'] = 43

# Fare에 값에 평균 값을 넣어준다.
test_data.loc[test_data.Fare.isnull(),'Fare'] = test_data['Fare'].mean()

In [506]:
# KNN 모델 적용에 앞서, Fare의 범위를 나눠준다.
combine = [train_data, test_data]

for dataset in combine:
    dataset.loc[ dataset['Fare'] <= 7.91, 'Fare'] = 0
    dataset.loc[(dataset['Fare'] > 7.91) & (dataset['Fare'] <= 14.454), 'Fare'] = 1
    dataset.loc[(dataset['Fare'] > 14.454) & (dataset['Fare'] <= 31), 'Fare']   = 2
    dataset.loc[ dataset['Fare'] > 31, 'Fare'] = 3
    dataset['Fare'] = dataset['Fare'].astype(int)

In [507]:
# KNN 모델 적용에 앞서, 성별 값을 0과 1로 변경을 한다.
train_data['Sex']=train_data['Sex'].replace('male', 0)
train_data['Sex']=train_data['Sex'].replace('female', 1)

test_data['Sex']=train_data['Sex'].replace('male', 0)
test_data['Sex']=train_data['Sex'].replace('female', 1)

In [508]:
label = train_data.iloc[0:890,1] # 훈련 데이터의 생존 데이터
data = train_data.iloc[0:890,[2,4,5,9]] # 훈련 데이터의 P class, Age, Sex, Fare 값
testset = test_data.iloc[0:418,[1,3,4,8]] # 테스트 데이터의 P class, Age, Sex, Fare 값

In [509]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier

train_input, test_input, train_target, test_target = train_test_split(data, label, test_size=0.3, random_state=7)

kn = KNeighborsClassifier(n_neighbors=1)
kn.fit(data,label)

predictions = kn.predict(test_input)
# test_input 특성을 통해 예측한 값 predictions!
print(predictions)

In [510]:
from sklearn.metrics import accuracy_score
# 테스트 데이터와 예측 값의 정확도를 얻을 수 있다.
print(accuracy_score(test_target,predictions))

result = kn.predict(testset)

# CSV 파일 출력단
output = pd.DataFrame({'PassengerId' : test_data.PassengerId, 'Survived': result})
output.to_csv("KNN_3.csv", index = False)
print("complete!")